<a href="https://colab.research.google.com/github/eispoohw/CS493-Math-Methods-in-ML/blob/main/lab%203.5-3.8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Предсказание цен на недвижимость

Компании по продаже недвижимости оценивают её стоимость, используя методы машинного обучения. 
Задача оценки цены на недвижимость также может быть нужна для выявления наиболее выгодных предложений, или, например, для выявления аномально низких цен на объект недвижимости.

**Постановка задачи анализа данных** 

Целью данной задачи является прогнозирование стоимости домов в округе Кинг (штат Вашингтон, США) с помощью построения регрессионных моделей и их анализа. Набор данных состоит из цен на дома в округе Кинг, проданных в период с мая 2014 года по май 2015 года. Данные опубликованы в открытом доступе на платформе Kaggle. 

## План анализа данных (data mining):

  1. Загрузить данные для обучения
  2. Обработать данные перед обучением модели
  3. Обучить модель на обучающей выборке
  4. Загрузить и предобработать данные для тестирования
  5. Провалидировать модель на тестовой выборке
  6. Проанализировать важность признаков и не забыть про постановку задачи

## 1. Загрузить данные для обучения

**Шаг 1.1. Загружаем библиотеки** 

Библиотека **warnings** отвечает за то, какие предупреждения (warnings) о работе будут выводиться пользователю. 
FutureWarning - предупреждения о том, как изменится работа библиотек в будущих версиях.
Поэтому такие предупреждения мы будем игнорировать.
Чтобы включить режим игнорирования мы отбираем все предупреждения из категории FutureWarning и выбираем для них действия 'ignore'.
Это делается вызовом функции simplefilter c задание двух атрибутов: действия action и категории предупреждений category.

In [1]:
import pandas as pd # загружаем библиотеку и для простоты обращения в коде называем её сокращенно pd
import numpy as np
import matplotlib.pyplot as plt # загружаем библиотеку и для простоты обращения в коде называем её сокращенно plt
# указываем, чтобы картинки отображались прямо в ноутбуке 
%matplotlib inline 
plt.style.use('ggplot')
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

**Шаг 1.2. Загрузим данные**

In [2]:
%%capture
!wget https://www.dropbox.com/s/afwb0tnqm9izxha/predict_house_price_training_data.xlsx
!wget https://www.dropbox.com/s/sur2avqf4n5f4az/predict_house_price_test_data.xlsx

In [3]:
training_data = pd.read_excel('predict_house_price_training_data.xlsx') # загружаем таблицу в переменную training_data

In [4]:
training_data.head()

,Целевая.Цена,Спальни,Ванные,Жилая площадь,Общая площадь,Количество этажей,Вид на воду,Просмотрены ранее,Состояние,Оценка риелтора,Площадь без подвала,Площадь подвала,Год постройки,Год реновации,Широта,Долгота
0,830000,5,3.50,3490,21780,2.0,0,0,3,8,3490,0,1996,0,47.6707,-122.144
1,385000,4,1.75,2360,7620,1.0,0,0,4,7,1180,1180,1955,0,47.5278,-122.345
2,610000,6,2.75,2040,8560,1.0,0,2,4,7,1100,940,1961,0,47.6160,-122.115
3,550000,3,1.75,1940,8376,1.0,0,0,4,8,1290,650,1963,0,47.5586,-122.173
4,1300000,3,2.75,3450,5350,1.5,0,3,4,9,2590,860,1925,0,47.6389,-122.407


**Шаг 1.3. Посмотрим на размеры загруженной таблицы**, у которой мы видели только первые 5 строк.

Для этого вызываем поле **shape** у нашей переменной *training_data*. Поле вызывается также как метод, но в конце скобки не ставятся, так как для поля не предусмотрена передача аргументов.  

In [5]:
training_data.shape

(15129, 16)

# 2. Обработать данные перед обучением модели

**Шаг 2.1. Проверяем данные на наличие пропусков и типов переменных**

Начнем с проверки общей информации о данных.
Для того чтобы это сделать, нужно обратиться вызвать у переменной *training_data* метод **info()**.

Напомним, что в конце необходимо поставить скобочки.

In [6]:
training_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15129 entries, 0 to 15128
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Целевая.Цена         15129 non-null  int64  
 1   Спальни              15129 non-null  int64  
 2   Ванные               15129 non-null  float64
 3   Жилая площадь        15129 non-null  int64  
 4   Общая площадь        15129 non-null  int64  
 5   Количество этажей    15129 non-null  float64
 6   Вид на воду          15129 non-null  int64  
 7   Просмотрены ранее    15129 non-null  int64  
 8   Состояние            15129 non-null  int64  
 9   Оценка риелтора      15129 non-null  int64  
 10  Площадь без подвала  15129 non-null  int64  
 11  Площадь подвала      15129 non-null  int64  
 12  Год постройки        15129 non-null  int64  
 13  Год реновации        15129 non-null  int64  
 14  Широта               15129 non-null  float64
 15  Долгота              15129 non-null 

**Шаг 2.2. Работаем с целевой переменной**

*Какая переменная целевая?*

В данном случае по условию задачи мы должны прогнозировать стоимость, поэтому целевая переменная - это цена. 

In [7]:
target_variable_name = 'Целевая.Цена'
training_values = training_data[target_variable_name]
training_points = training_data.drop(target_variable_name, axis=1)

##   3. Обучить модель на обучающей выборке

**Выбираем метод, который будем использовать**

Проще всего начать с простых методов. 

На выбор метода для построения модели влияет набор признаков, размер выборки, интуиция про то, какая связь между входными переменными и целевой. Но часто решение принимается исходя из того, какая модель сработала лучше.

In [8]:
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
training_points_scale = scaler.fit_transform(training_points)

linear_regression_model = linear_model.LinearRegression() # создаем модель
linear_regression_model.fit(training_points_scale, training_values)

LinearRegression()

## Задание 3.5

Реализуйте настройку параметров модели не с помощью встроенной функции, а с помощью своей реализации.

Настроить параметры нужно либо с помощью **стохастического градиентного спуска** либо с помощью **mini-batch градиентного спуска**. 
Нужно использовать регуляризацию **L1, L2 или Elastic Net**. 

$$ Q(w) = \frac{1}{rows}\sum_{j=1}^{rows} {\left( {w_0 + \sum_{i=1}^{features}{w_ix^{j}_i} - y^j}\right)^2} + \alpha \sum_{k=0}^{features} w^2_k $$

In [9]:
import numpy as np

def stochastic_descent(X, Y, bound=1000, alpha=0.001, step=lambda k: 1/k):
    R, F = X.shape # R - rows count, F - features
    X1 = np.concatenate((np.ones((R,1)), X), axis=1)
    W = np.ones(F+1)
    for k in range(1, bound):
        idx = np.random.randint(0, R+1)
        curr_y = X1[idx] @ W - Y[idx]
        diff = step(k) * (X1[idx] * curr_y - alpha * W)
        W = W - diff
    return W


sd = stochastic_descent(training_points_scale, training_values)
bias, weights = sd[0], sd[1:]

wb = pd.DataFrame()
wb = wb.append([np.round(weights)])
wb = wb.append([np.round(linear_regression_model.coef_)])
wb['bias'] = [np.round(bias), np.round(linear_regression_model.intercept_)]
wb 

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,bias
0,9571.0,247316.0,-4373.0,-46891.0,-218488.0,156382.0,-75216.0,48963.0,245134.0,52094.0,-107065.0,-37961.0,15752.0,73001.0,-92707.0,534260.0
0,-31501.0,33056.0,86664.0,-1834.0,-3417.0,50962.0,38203.0,20958.0,116577.0,82692.0,24394.0,-71827.0,9865.0,79048.0,-15305.0,538795.0


# 4. Загрузить и предобработать данные для тестирования

**Шаг 4.1. Загрузим и проанализируем тестовые данные.**

Так как данные в формате xlsx (Excel), мы будем использовать специальную функцию
из библиотеки pandas для загрузки таких данных **read_excel**.

В функции передаем один атрибут: название файла, в котором находится таблица с данными.

In [10]:
test_data = pd.read_excel('predict_house_price_test_data.xlsx')
test_data.head()

,Целевая.Цена,Спальни,Ванные,Жилая площадь,Общая площадь,Количество этажей,Вид на воду,Просмотрены ранее,Состояние,Оценка риелтора,Площадь без подвала,Площадь подвала,Год постройки,Год реновации,Широта,Долгота
0,260000,3,1.00,1300,10139,1.0,0,0,3,7,1300,0,1962,2007,47.3427,-122.087
1,734500,4,2.75,3280,6845,2.0,0,0,3,10,3280,0,2003,0,47.7042,-122.107
2,325000,1,1.00,1220,12426,1.0,0,4,4,6,1220,0,1946,0,47.4047,-122.331
3,1990000,3,2.50,2880,13500,1.0,0,4,5,8,1520,1360,1950,0,47.6281,-122.216
4,315000,3,2.00,1300,3731,1.0,0,0,3,7,900,400,1993,0,47.5374,-122.270


**Шаг 4.2. Отделяем целевую переменную**

In [11]:
test_values = test_data[target_variable_name]
test_points = test_data.drop(target_variable_name, axis=1)

# 5. Провалидировать модель на тестовой выборке

**Шаг 5.1. Сравнение моделей.**

Теперь мы готовы сравнить качество двух моделей! 😎

## Задание 3.6 Какая модель лучше?

Получим прогнозы целевой переменной на тестовых данных для модели линейной регрессии из sklearn и для своей реализации. 



In [12]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error

test_points_scale = scaler.transform(test_points)

test_predictions_linear = linear_regression_model.predict(test_points_scale)

compare = pd.DataFrame()
compare = compare.append({
    'name': 'sklearn',
    'MAE': mean_absolute_error(test_predictions_linear, test_values),
    'RMSE': np.sqrt(mean_squared_error(test_predictions_linear, test_values))
}, ignore_index=True)

compare = compare.append({
    'name': 'SGD',
    'MAE': mean_absolute_error(np.matmul(test_points_scale, weights) + bias, test_values),
    'RMSE': np.sqrt(mean_squared_error(np.matmul(test_points_scale, weights) + bias, test_values))
}, ignore_index=True)

compare = compare.append({
    'name': 'fraction',
    'MAE': compare['MAE'][0] / compare['MAE'][1],
    'RMSE': compare['RMSE'][0] / compare['RMSE'][1]
}, ignore_index=True)

compare

,name,MAE,RMSE
0,sklearn,126852.512550,201883.242903
1,SGD,229646.036891,322669.450123
2,fraction,0.552383,0.625666


## 6. Выявление важных признаков

Признаки, используемые для обучения модели, оказывают большое влияние на качество результатов. Неинформативные или слабо информативные признаки могут снизить качество модели, использование избыточных признаков в модели замедляет скорость ее работы.

Для того, чтобы отобрать признаки нам следует ответить на следующие вопросы: 

    1. Все ли признаки в наших данных заполненны разумными значениями?
    2. Какие признаки будут больше всего влиять на значение целевой переменной?
    3. Какие дополнительные признаки имело бы смысл добавить в список входных?

**6.1. Разглядывание значений признаков**

Для того чтобы ответить на первый вопрос, нам необходимо получить некоторую нашу экспертную оценку данных. В наших данных нет пропусков, однако иногда пропуски предварительно заполняют другими значениями (0, 999 и другие) и их можно увидеть, только внимательно посмотрев на данные. 

Для того, чтобы это сделать, нужно вызвать у переменной *training_points* метод **head(10)**, который выводит первые 10 строк таблицы.

In [13]:
training_points.head(10)

,Спальни,Ванные,Жилая площадь,Общая площадь,Количество этажей,Вид на воду,Просмотрены ранее,Состояние,Оценка риелтора,Площадь без подвала,Площадь подвала,Год постройки,Год реновации,Широта,Долгота
0,5,3.50,3490,21780,2.0,0,0,3,8,3490,0,1996,0,47.6707,-122.144
1,4,1.75,2360,7620,1.0,0,0,4,7,1180,1180,1955,0,47.5278,-122.345
2,6,2.75,2040,8560,1.0,0,2,4,7,1100,940,1961,0,47.6160,-122.115
3,3,1.75,1940,8376,1.0,0,0,4,8,1290,650,1963,0,47.5586,-122.173
4,3,2.75,3450,5350,1.5,0,3,4,9,2590,860,1925,0,47.6389,-122.407
5,3,2.25,2300,9914,2.0,0,0,4,8,2300,0,1980,0,47.5677,-122.086
6,3,2.50,2770,8820,1.0,0,0,3,7,1900,870,1980,2004,47.3685,-122.048
7,2,1.75,1650,7500,1.0,0,0,4,7,1000,650,1959,0,47.6871,-122.207
8,1,1.00,580,1799,1.0,0,0,3,7,580,0,1908,2005,47.6829,-122.375
9,2,1.00,900,3400,1.0,0,0,5,6,900,0,1905,0,47.5269,-122.314


Рассмотрим столбец "Год реновации". Он содержит много нулей и мало реальных данных о годах, что как раз свидетельствует о том, что в этих данных изначально были пропуски, а к нам они попали уже заполненные нулями. 

Мы можем посмотреть количество уникальных значений и сколько раз эти значения встречаются в этом столбце. Для этого вызываем метод **value_counts()** у нашего столбца *training_points['Год реновации']*

In [14]:
training_points['Год реновации'].value_counts()

0       14490
2014       63
2013       31
2000       28
2003       24
        ...  
1948        1
1950        1
1953        1
1934        1
1955        1
Name: Год реновации, Length: 67, dtype: int64

Как видно, нулей существенно больше (14490), чем любых других значений (639), поэтому можем сделать вывод, что этот признак не информативен и может негативно повлиять на качество модели.

**6.2. Какие признаки самые важные**

Ручной перебор признаков невозможен, если их тысячи, кроме того, часто даже эксперты не могут сказать, использование каких признаков в модели даст прирост качества.
Кроме ручного перебора всех признаков и их визуального анализа, можно использовать оценку значимости признаков с помощью обученных моделей. 

## Задание 3.7 

In [15]:
sorted_features = pd.DataFrame(np.abs(linear_regression_model.coef_.reshape(1, -1)), columns=training_points.columns).T.sort_values(by=0, ascending=False).reset_index()
sorted_features

,index,0
0,Оценка риелтора,116577.366181
1,Жилая площадь,86664.293134
2,Площадь без подвала,82691.723728
3,Широта,79047.942776
4,Год постройки,71827.278259
5,Вид на воду,50962.118212
6,Просмотрены ранее,38203.271673
7,Ванные,33055.646855
8,Спальни,31500.701385
9,Площадь подвала,24394.133310


In [16]:
sorted_features['Название признака'] = sorted_features['index']
sorted_features['Важность признака'] = sorted_features.index + 1

feature_importance = sorted_features[['Название признака', 'Важность признака']]
feature_importance

,Название признака,Важность признака
0,Оценка риелтора,1
1,Жилая площадь,2
2,Площадь без подвала,3
3,Широта,4
4,Год постройки,5
5,Вид на воду,6
6,Просмотрены ранее,7
7,Ванные,8
8,Спальни,9
9,Площадь подвала,10


## Задание 3.8. 

Напомню исходную постановку задачи. Мы говорили о том, что оценка цены на недвижимость может быть нужна для выявления аномально низких цен на объект недвижимости. Давайте попробуем найти такие в датасете.

In [17]:
anomalous_diff = 1000000

diff_y = test_data
diff_y['predicted'] = np.round(test_predictions_linear)
diff_y['difference'] = np.round(test_data[target_variable_name] - test_predictions_linear)
diff_y[[target_variable_name, 'predicted', 'difference']][diff_y['difference'] > anomalous_diff].sort_values(by='difference', ascending=False)

,Целевая.Цена,predicted,difference
5627,5350000,2261467.0,3088533.0
5462,5110000,2886123.0,2223877.0
2592,3650000,1486444.0,2163556.0
4175,4000000,1934876.0,2065124.0
5202,3640000,1680277.0,1959723.0
2068,3200000,1497726.0,1702274.0
5718,3800000,2146280.0,1653720.0
483,2950000,1422635.0,1527365.0
5597,2850000,1353982.0,1496018.0
4811,2900000,1461435.0,1438565.0
